In [1]:
import numpy as np
import sys
import os
import random 
import pandas as pd
import torch
from torch import nn, cuda, optim
from torchvision import models,transforms,datasets
from torch.utils.data import DataLoader,random_split, dataset
import seaborn as sns
import torch.nn.functional as F
import torch.nn as nn
from math import ceil
import shutil
import time
import copy
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_excel("/content/drive/MyDrive/DCD_2022/음식분류 AI 데이터 영양DB.xlsx", engine = "openpyxl")

In [4]:
df = df.replace('-', 0)

In [5]:
device = torch.device('cuda:0' if cuda.is_available() else 'cpu')

In [6]:
class SEBlock(nn.Module):
    def __init__(self, in_channels, r=16):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d((1,1))
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels // r),
            nn.ReLU(),
            nn.Linear(in_channels // r, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.squeeze(x)
        x = x.view(x.size(0), -1) 
        x = self.excitation(x)
        x = x.view(x.size(0), x.size(1), 1, 1)
        return x

# Depthwise Separable Convolution
class Depthwise(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.depthwise = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU6(),
        )

        self.pointwise = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU6(),
        )

        self.seblock = SEBlock(out_channels)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.seblock(x) * x
        return x


# BasicConv2d
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, **kwargs),
            nn.BatchNorm2d(out_channels),
            nn.ReLU6()
        )

    def forward(self, x):
        x = self.conv(x)
        return x


# MobileNetV1
class MobileNet(nn.Module):
    def __init__(self, width_multiplier, num_classes=83, init_weights=True):
        super().__init__()
        self.init_weights=init_weights
        alpha = width_multiplier

        self.conv1 = BasicConv2d(3, int(32*alpha), 3, stride=2, padding=1)
        self.conv2 = Depthwise(int(32*alpha), int(64*alpha), stride=1)
        # down sample
        self.conv3 = nn.Sequential(
            Depthwise(int(64*alpha), int(128*alpha), stride=2),
            Depthwise(int(128*alpha), int(128*alpha), stride=1)
        )
        # down sample
        self.conv4 = nn.Sequential(
            Depthwise(int(128*alpha), int(256*alpha), stride=2),
            Depthwise(int(256*alpha), int(256*alpha), stride=1)
        )
        # down sample
        self.conv5 = nn.Sequential(
            Depthwise(int(256*alpha), int(512*alpha), stride=2),
            Depthwise(int(512*alpha), int(512*alpha), stride=1),
            Depthwise(int(512*alpha), int(512*alpha), stride=1),
            Depthwise(int(512*alpha), int(512*alpha), stride=1),
            Depthwise(int(512*alpha), int(512*alpha), stride=1),
            Depthwise(int(512*alpha), int(512*alpha), stride=1),
        )
        # down sample
        self.conv6 = nn.Sequential(
            Depthwise(int(512*alpha), int(1024*alpha), stride=2)
        )
        # down sample
        self.conv7 = nn.Sequential(
            Depthwise(int(1024*alpha), int(1024*alpha), stride=2)
        )

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.linear = nn.Linear(int(1024*alpha), num_classes)

        # weights initialization
        if self.init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

    # weights initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def mobilenet(alpha=1, num_classes=83):
    return MobileNet(alpha, num_classes)

In [7]:
food_model = mobilenet(num_classes=83)
vol_model = mobilenet(num_classes=5)
# Load state_dict
food_model.load_state_dict(torch.load('/content/drive/MyDrive/Types of food_SENet_weight.pt'))
vol_model.load_state_dict(torch.load('/content/drive/MyDrive/Amount of food_SENet_weight.pt'))

<All keys matched successfully>

In [8]:
food_class_names = ['갈비탕',
 '갈치조림',
 '감자탕',
 '감자튀김(스틱형)',
 '감자튀김(웨지감자)',
 '곱창전골',
 '김밥',
 '김치볶음밥',
 '깻잎장아찌',
 '꽁치조림',
 '꽃게탕',
 '녹두빈대떡',
 '달걀말이',
 '닭갈비',
 '닭볶음탕',
 '닭죽',
 '닭튀김',
 '돈가스',
 '돼지갈비',
 '두부구이',
 '두부김치',
 '떡갈비',
 '떡만둣국',
 '떡볶이',
 '라볶이',
 '류산슬덮밥',
 '매운탕',
 '문어숙회',
 '물회',
 '미소된장국',
 '병어찜',
 '부대찌개',
 '삼계탕',
 '삼선볶음밥',
 '삼선짬뽕',
 '새우볶음밥',
 '새우튀김',
 '소고기김밥',
 '소고기무국',
 '수제비',
 '순대국',
 '순대볶음',
 '순두부찌개',
 '시금치나물',
 '시래기된장국',
 '쌀밥',
 '알밥',
 '알탕',
 '양념게장',
 '어묵볶음',
 '영양돌솥밥',
 '오곡밥',
 '오므라이스(사각배달)',
 '오므라이스(접시)',
 '오징어볶음',
 '오징어젓갈',
 '육개장',
 '일식우동',
 '자장면',
 '잡탕밥',
 '전주비빔밥',
 '전주콩나물국밥',
 '제육덮밥',
 '쫄면',
 '참치마요삼각김밥',
 '채소튀김',
 '총각김치',
 '치킨윙',
 '카레라이스',
 '코다리조림',
 '콘스프',
 '콩국수',
 '콩밥',
 '크림소스스파게티',
 '토마토소스스파게티',
 '파전',
 '해물찜',
 '햄버거스테이크',
 '호박죽',
 '회냉면',
 '회덮밥',
 '훈제연어',
 '훈제오리']

vol_class_names = ["Q1","Q2", "Q3", "Q4","Q5"]

In [9]:
# Create the preprocessing transformation here
val_transform = transforms.Compose([transforms.Resize((224,224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


food_pred = []
vol_pred = []

today_food = ["/content/drive/MyDrive/DCD_2022/음식 이미지 및 영양정보 텍스트/val_cropped/쌀밥/side_밥류_원형배달_쌀밥_Q3_00033.JPG",
    "/content/drive/MyDrive/DCD_2022/음식 이미지 및 영양정보 텍스트/val_cropped/갈비탕/side_건더기국류_뚝배기_갈비탕_Q5_00028.JPG", 
    "/content/drive/MyDrive/DCD_2022/음식 이미지 및 영양정보 텍스트/val_cropped/갈치조림/side_생선조림_냄비_갈치조림_Q1 00001.JPG"]

# Set model to eval
food_model.eval()
vol_model.eval()

for i in today_food:
  img = Image.open(i) 
  # Transform
  input = val_transform(img)

  # unsqueeze batch dimension, in case you are dealing with a single image
  input = input.unsqueeze(0)

  # Get prediction
  food_output = food_model(input)
  food_output = (torch.max(torch.exp(food_output), 1)[1]).data.cpu().numpy()
  vol_output = vol_model(input)
  vol_output = (torch.max(torch.exp(vol_output), 1)[1]).data.cpu().numpy()

  food_pred.extend(food_output) # Save Prediction
  vol_pred.extend(vol_output)

  food_name = np.array(food_class_names)[food_pred]
  vol_name = np.array(vol_class_names)[vol_pred]
  
print(food_name)
print(vol_name)

['쌀밥' '갈비탕' '갈치조림']
['Q3' 'Q5' 'Q1']


In [11]:
lst = []
weight = []
kal = []
tan = []
dang = []
gi = []
dan = []
cal = []
inn = []
na = []
kalum = []
mg = []
F = []
A = []
col = []
tran = []
total = []

for i in range(len(today_food)):
  if len(df.loc[(df['음 식 명'] == food_name[i])]) == 1:
    a = df.index[(df['음 식 명']) == food_name[i]].tolist()
    lst.append(a[0])
    if vol_name[i] == "Q1":
      weight.append(round(df.at[lst[i], '중량(g)'] * 0.25,2))
      kal.append(round(df.at[lst[i], '에너지(kcal)'] * 0.25,2))
      tan.append(round(df.at[lst[i], '탄수화물(g)'] * 0.25,2))
      dang.append(round(df.at[lst[i], '당류(g)'] * 0.25,2))
      gi.append(round(df.at[lst[i], '지방(g)'] * 0.25,2))
      dan.append(round(df.at[lst[i], '단백질(g)'] * 0.25,2))
      cal.append(round(df.at[lst[i], '칼슘(mg)'] * 0.25,2))
      inn.append(round(df.at[lst[i], '인(mg)'] * 0.25,2))
      na.append(round(df.at[lst[i], '나트륨(mg)'] * 0.25,2))
      kalum.append(round(df.at[lst[i], '칼륨(mg)'] * 0.25,2))
      mg.append(round(df.at[lst[i], '마그네슘(mg)'] * 0.25,2))
      F.append(round(df.at[lst[i], '철(mg)'] * 0.25,2))
      A.append(round(df.at[lst[i], '아연(mg)'] * 0.25,2))
      col.append(round(df.at[lst[i], '콜레스테롤(mg)'] * 0.25,2))
      tran.append(round(df.at[lst[i], '트랜스지방(g)'] * 0.25,2))
    elif vol_name[i] == "Q2":
      weight.append(round(df.at[lst[i], '중량(g)'] * 0.5,2))
      kal.append(round(df.at[lst[i], '에너지(kcal)'] * 0.5,2))
      tan.append(round(df.at[lst[i], '탄수화물(g)'] * 0.5,2))
      dang.append(round(df.at[lst[i], '당류(g)'] * 0.5,2))
      gi.append(round(df.at[lst[i], '지방(g)'] * 0.5,2))
      dan.append(round(df.at[lst[i], '단백질(g)'] * 0.5,2))
      cal.append(round(df.at[lst[i], '칼슘(mg)'] * 0.5,2))
      inn.append(round(df.at[lst[i], '인(mg)'] * 0.5,2))
      na.append(round(df.at[lst[i], '나트륨(mg)'] * 0.5,2))
      kalum.append(round(df.at[lst[i], '칼륨(mg)'] * 0.5,2))
      mg.append(round(df.at[lst[i], '마그네슘(mg)'] * 0.5,2))
      F.append(round(df.at[lst[i], '철(mg)'] * 0.5,2))
      A.append(round(df.at[lst[i], '아연(mg)'] * 0.5,2))
      col.append(round(df.at[lst[i], '콜레스테롤(mg)'] * 0.5,2))
      tran.append(round(df.at[lst[i], '트랜스지방(g)'] * 0.5,2))
    elif vol_name[i] == "Q3":
      weight.append(round(df.at[lst[i], '중량(g)'] * 0.75,2))
      kal.append(round(df.at[lst[i], '에너지(kcal)'] * 0.75,2))
      tan.append(round(df.at[lst[i], '탄수화물(g)'] * 0.75,2))
      dang.append(round(df.at[lst[i], '당류(g)'] * 0.75,2))
      gi.append(round(df.at[lst[i], '지방(g)'] * 0.75,2))
      dan.append(round(df.at[lst[i], '단백질(g)'] * 0.75,2))
      cal.append(round(df.at[lst[i], '칼슘(mg)'] * 0.75,2))
      inn.append(round(df.at[lst[i], '인(mg)'] * 0.75,2))
      na.append(round(df.at[lst[i], '나트륨(mg)'] * 0.75,2))
      kalum.append(round(df.at[lst[i], '칼륨(mg)'] * 0.75,2))
      mg.append(round(df.at[lst[i], '마그네슘(mg)'] * 0.75,2))
      F.append(round(df.at[lst[i], '철(mg)'] * 0.75,2))
      A.append(round(df.at[lst[i], '아연(mg)'] * 0.75,2))
      col.append(round(df.at[lst[i], '콜레스테롤(mg)'] * 0.75,2))
      tran.append(round(df.at[lst[i], '트랜스지방(g)'] * 0.75,2))
    elif vol_name[i] == "Q4":
      weight.append(round(df.at[lst[i], '중량(g)'] * 1,2))
      kal.append(round(df.at[lst[i], '에너지(kcal)'] * 1,2))
      tan.append(round(df.at[lst[i], '탄수화물(g)'] * 1,2))
      dang.append(round(df.at[lst[i], '당류(g)'] * 1,2))
      gi.append(round(df.at[lst[i], '지방(g)'] * 1,2))
      dan.append(round(df.at[lst[i], '단백질(g)'] * 1,2))
      cal.append(round(df.at[lst[i], '칼슘(mg)'] * 1,2))
      inn.append(round(df.at[lst[i], '인(mg)'] * 1,2))
      na.append(round(df.at[lst[i], '나트륨(mg)'] * 1,2))
      kalum.append(round(df.at[lst[i], '칼륨(mg)'] * 1,2))
      mg.append(round(df.at[lst[i], '마그네슘(mg)'] * 1,2))
      F.append(round(df.at[lst[i], '철(mg)'] * 1,2))
      A.append(round(df.at[lst[i], '아연(mg)'] * 1,2))
      col.append(round(df.at[lst[i], '콜레스테롤(mg)'] * 1,2))
      tran.append(round(df.at[lst[i], '트랜스지방(g)'] * 1,2))
    else:
      weight.append(round(df.at[lst[i], '중량(g)'] * 1.25,2))
      kal.append(round(df.at[lst[i], '에너지(kcal)'] * 1.25,2))
      tan.append(round(df.at[lst[i], '탄수화물(g)'] * 1.25,2))
      dang.append(round(df.at[lst[i], '당류(g)'] * 1.25,2))
      gi.append(round(df.at[lst[i], '지방(g)'] * 1.25,2))
      dan.append(round(df.at[lst[i], '단백질(g)'] * 1.25,2))
      cal.append(round(df.at[lst[i], '칼슘(mg)'] * 1.25,2))
      inn.append(round(df.at[lst[i], '인(mg)'] * 1.25,2))
      na.append(round(df.at[lst[i], '나트륨(mg)'] * 1.25,2))
      kalum.append(round(df.at[lst[i], '칼륨(mg)'] * 1.25,2))
      mg.append(round(df.at[lst[i], '마그네슘(mg)'] * 1.25,2))
      F.append(round(df.at[lst[i], '철(mg)'] * 1.25,2))
      A.append(round(df.at[lst[i], '아연(mg)'] * 1.25,2))
      col.append(round(df.at[lst[i], '콜레스테롤(mg)'] * 1.25,2))
      tran.append(round(df.at[lst[i], '트랜스지방(g)'] * 1.25,2))
    print(f"음식{i+1} : {food_name[i]} / 중량(g) : {weight[i]} / 에너지(kcal) : {kal[i]} / 탄수화물(g) : {tan[i]} / 당류(g) : {dang[i]} / 지방(g) : {gi[i]} / 단백질(g) : {dan[i]} / 칼슘(mg) : {cal[i]} / 인(mg) : {inn[i]} \n나트륨(mg) : {na[i]} / 칼륨(mg) : {kalum[i]} / 마그네슘(mg) : {mg[i]} / 철(mg) : {F[i]} / 아연(mg) : {A[i]} / 콜레스테롤(mg) {col[i]} / 트랜스지방(g) : {tran[i]}\n" )
print(f"총 영양성분 / 중량(g) : {round(sum(weight),2)} / 에너지(kcal) : {round(sum(kal),2)} / 탄수화물(g) : {round(sum(tan),2)} / 당류(g) : {round(sum(dang),2)} / 지방(g) : {round(sum(gi),2)} / 단백질(g) : {round(sum(dan),2)} / 칼슘(mg) : {round(sum(cal),2)} / 인(mg) : {round(sum(inn),2)} \n나트륨(mg) : {round(sum(na),2)} / 칼륨(mg) : {round(sum(kalum),2)} / 마그네슘(mg) : {round(sum(mg),2)} / 철(mg) : {round(sum(F),2)} / 아연(mg) : {round(sum(A),2)} / 콜레스테롤(mg) {round(sum(col),2)} / 트랜스지방(g) : {round(sum(tran),2)}\n" )

음식1 : 쌀밥 / 중량(g) : 157.5 / 에너지(kcal) : 251.1 / 탄수화물(g) : 55.28 / 당류(g) : 0.0 / 지방(g) : 0.34 / 단백질(g) : 4.32 / 칼슘(mg) : 2.7 / 인(mg) : 94.5 
나트륨(mg) : 44.55 / 칼륨(mg) : 110.02 / 마그네슘(mg) : 0.0 / 철(mg) : 0.27 / 아연(mg) : 1.01 / 콜레스테롤(mg) 0.0 / 트랜스지방(g) : 0.0

음식2 : 갈비탕 / 중량(g) : 750.0 / 에너지(kcal) : 300.5 / 탄수화물(g) : 10.26 / 당류(g) : 0.0 / 지방(g) : 17.92 / 단백질(g) : 23.34 / 칼슘(mg) : 69.14 / 인(mg) : 255.71 
나트륨(mg) : 2110.54 / 칼륨(mg) : 696.14 / 마그네슘(mg) : 0.0 / 철(mg) : 3.3 / 아연(mg) : 5.08 / 콜레스테롤(mg) 237.3 / 트랜스지방(g) : 0.0

음식3 : 갈치조림 / 중량(g) : 25.0 / 에너지(kcal) : 24.85 / 탄수화물(g) : 1.37 / 당류(g) : 0.12 / 지방(g) : 0.98 / 단백질(g) : 2.67 / 칼슘(mg) : 9.67 / 인(mg) : 32.44 
나트륨(mg) : 115.96 / 칼륨(mg) : 75.75 / 마그네슘(mg) : 1.54 / 철(mg) : 0.29 / 아연(mg) : 0.15 / 콜레스테롤(mg) 10.5 / 트랜스지방(g) : 0.0

총 영양성분 / 중량(g) : 932.5 / 에너지(kcal) : 576.45 / 탄수화물(g) : 66.91 / 당류(g) : 0.12 / 지방(g) : 19.24 / 단백질(g) : 30.33 / 칼슘(mg) : 81.51 / 인(mg) : 382.65 
나트륨(mg) : 2271.05 / 칼륨(mg) : 881.91 / 마그네슘(mg) : 1.54 / 철(mg) : 3.86 / 아연(m

In [14]:
  """
  필요량 / 권장 / 상한
  20대 남자 
  에너지 :  2600 /- /- Kcal, 탄수화물 : 100/ 130/- g, 단백질 : 50/ 65/-g
  지방 : -/-/51
  칼슘 : 650 / 800 / 2500 mg, 인 : 580/700/3500mg, 마그네슘 : 300 / 360 / -
  나트륨 : -/-/1500mg, 칼륨 : -/-/3500, 철 : 8 / 10 / 45mg, 아연 : 9 / 10 / 35mg

  20대 여자
  에너지 :  2000 /- /- Kcal, 탄수화물 : 100/ 130/- g, 단백질 : 45/ 55/-g
  칼슘 : 550 / 700 / 2500 mg, 인 : 580/700/3500mg, 마그네슘 : 230 / 280 / -
  나트륨 : -/-/1500mg, 칼륨 : -/-/3500, 철 : 11 / 14 / 45, 아연 : 7 / 8 / 35

  출처 보건복지부 2020년 발표자료
  """

'\n필요량 / 권장 / 상한\n20대 남자 \n에너지 :  2600 /- /- Kcal, 탄수화물 : 100/ 130/- g, 단백질 : 50/ 65/-g\n지방 : -/-/51\n칼슘 : 650 / 800 / 2500 mg, 인 : 580/700/3500mg, 마그네슘 : 300 / 360 / -\n나트륨 : -/-/1500mg, 칼륨 : -/-/3500, 철 : 8 / 10 / 45mg, 아연 : 9 / 10 / 35mg\n\n20대 여자\n에너지 :  2000 /- /- Kcal, 탄수화물 : 100/ 130/- g, 단백질 : 45/ 55/-g\n칼슘 : 550 / 700 / 2500 mg, 인 : 580/700/3500mg, 마그네슘 : 230 / 280 / -\n나트륨 : -/-/1500mg, 칼륨 : -/-/3500, 철 : 11 / 14 / 45, 아연 : 7 / 8 / 35\n\n출처 보건복지부 2020년 발표자료\n'